48. 名詞から根へのパスの抽出

文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した

In [2]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __str__(self):
        return f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}"

class Chunk:
    def __init__(self, morphs=None, dst=-1):
        self.morphs = morphs if morphs else []  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst  # 係り先文節インデックス番号
        self.srcs = []  # 係り元文節インデックス番号のリスト

    def __str__(self):
        morphs_surface = ''.join([morph.surface for morph in self.morphs])
        return f"Chunk: {morphs_surface}, dst: {self.dst}, srcs: {self.srcs}"

    def get_text(self):
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])

def parse_cabocha_output(parsed_file):
    sentences = []
    with open(parsed_file, 'r', encoding='utf-8') as f:
        chunks = {}
        chunk = None
        for line in f:
            if line == 'EOS\n':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                if chunks:
                    sorted_chunks = sorted(chunks.items())
                    sentence = [chunk for idx, chunk in sorted_chunks]
                    for idx, chunk in sorted_chunks:
                        if chunk.dst != -1:
                            sentence[chunk.dst].srcs.append(idx)
                    sentences.append(sentence)
                chunks = {}
                chunk = None
            elif line[0] == '*':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                cols = line.split(' ')
                chunk_idx = int(cols[1])
                dst = int(cols[2].rstrip('D'))
                chunk = Chunk(dst=dst)
            else:
                surface, feature = line.split('\t')
                features = feature.split(',')
                if len(features) >= 7:
                    morph = Morph(surface, features[6], features[0], features[1])
                    chunk.morphs.append(morph)
    return sentences

def extract_paths_to_root(parsed_file):
    sentences = parse_cabocha_output(parsed_file)
    paths = []
    for sentence in sentences:
        for idx, chunk in enumerate(sentence):
            if any(morph.pos == '名詞' for morph in chunk.morphs):
                path = []
                current_chunk = chunk
                while current_chunk is not None:
                    path.append(current_chunk.get_text())
                    if current_chunk.dst != -1:
                        current_chunk = sentence[current_chunk.dst]
                    else:
                        current_chunk = None
                paths.append(' -> '.join(path))
    return paths

# 指定解析結果ファイルのパス
parsed_file = 'ai.ja.txt.parsed'

# 名詞を含む文節から根へのパスを抽出
paths = extract_paths_to_root(parsed_file)

# 結果を表示
for path in paths:
    print(path)


人工知能 -> 語
じんこうちのう -> 語
AI -> エーアイとは -> 語
エーアイとは -> 語
計算 -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
概念と -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
コンピュータ -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
知能を -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語
計算機科学 -> の -> 一分野を -> 指す -> 語
一分野を -> 指す -> 語
語
言語の -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
理解や -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
人間に -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
コンピューターに -> 行わせる -> 技術または -> 研究分野とも -> される
技術または -> 研究分野とも -> される
計算機 -> コンピュータによる -> 情報処理システムの -> 実現に関する -> 研究分野とも -> される
コンピュータによる -> 情報処理システムの -> 実現に関する -> 研究分野とも -> される
知的な -> 情報処理システムの -> 実現に関する -> 研究分野とも